In [1]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI

## Creating the tools we need to help our agents do the job well
- These should be under a tools file

In [10]:
## FIXING GOOGLE SEARCH API CALL
import requests
import json

url = "https://google.serper.dev/search"

payload = json.dumps({
  "q": "Weather in thailand"
})
headers = {
  'X-API-KEY': 'fdd238e134ca2188f296f77e17adfd9368860fc8',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
# check if there is an organic key
def get_response(output):
  if 'organic' not in output.json():
        return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
  else:
        results = output.json()['organic']
        string = []
        for result in results[:4]:
          try:
            string.append('\n'.join([
                f"Title: {result['title']}", f"Link: {result['link']}",
                f"Snippet: {result['snippet']}", "\n-----------------"
            ]))
          except KeyError:
            next
  return '\n'.join(string)

print(get_response(response.text))

AttributeError: 'str' object has no attribute 'json'

In [2]:
# Search tools
'''
Refer to https://serper.dev/playground to get source code for the API calling or to get your key
'''
import json
import os
import requests
from dotenv import load_dotenv
from langchain.tools import tool

load_dotenv()
SERPER_KEY = os.environ.get('SERPER_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')


class SearchTools():

  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    num_results = 4
    # Set API endpoint
    url = "https://google.serper.dev/search"
    
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': SERPER_KEY,
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
      return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
      results = response.json()['organic']
      string = []
      for result in results[:num_results]:
        try:
          string.append('\n'.join([
              f"Title: {result['title']}", f"Link: {result['link']}",
              f"Snippet: {result['snippet']}", "\n-----------------"
          ]))
        except KeyError:
          next

      return '\n'.join(string)
    
    
from langchain.tools import tool
#calculator tools
class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        '''
        Useful for performing any mathematical calculations, like sum minus, multiplication,
        The input to this tool should be a mathematical expression, a couple of examples are '200x7' or '5000/2*10'
        '''
        try:
            return eval(operation)
        except SyntaxError:
            return "ERROR: Invalid syntax in mathematical expression"      

## Cheatsheet on how to start defining the agents


In [3]:
'''
Creating agents cheatsheet:
- Think like a boss, work backwards from the goal and think about what employees you want to hire
- Define the captain of the crew who orient the other agents towards the goal
- Define which experts the captain needs to communicate with and delegate tasks to.
- Build a top down approach

Goal:
- Create a 7 day itinerary with detailed per day plans., including budget packing suggestions and safety tips.

Captain:
- Expert travel agent

Employees:
- City selection expert
- Local tour guide expert

Notes:
- IMPORTANT: Agent should be *results* driven and have a clear goal in mind
- Role is their job title
- Goals should be actionable
- Backstory should be their resume
'''

'\nCreating agents cheatsheet:\n- Think like a boss, work backwards from the goal and think about what employees you want to hire\n- Define the captain of the crew who orient the other agents towards the goal\n- Define which experts the captain needs to communicate with and delegate tasks to.\n- Build a top down approach\n\nGoal:\n- Create a 7 day itinerary with detailed per day plans., including budget packing suggestions and safety tips.\n\nCaptain:\n- Expert travel agent\n\nEmployees:\n- City selection expert\n- Local tour guide expert\n\nNotes:\n- IMPORTANT: Agent should be *results* driven and have a clear goal in mind\n- Role is their job title\n- Goals should be actionable\n- Backstory should be their resume\n'

## Generating the custom agents that we need to perform our task

In [4]:
class TravelAgents:
    def __init__(self):
        self.OpenAIGPT35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, api_key=OPENAI_API_KEY)

    def expert_travel_agent(self):
        return Agent(
            role="Expert Travel Agent",
            # Do the back story last, because the backstory should support the goal
            backstory=dedent(f"""Expert in travel planning and logistics. I have decades of experience making travel itineraries."""),
            goal=dedent(f"""Create a 7 day travel itinerary with detailed per-day plans, 
                        include budget packing suggestions and safety tips.
                        """),
            tools=[SearchTools.search_internet,
                   CalculatorTools.calculate],
            allow_delegation=False,
            verbose=True,
            #CUSTOMISE THIS
            llm=self.OpenAIGPT35,
        )

    def city_selection_expert(self):
        return Agent(
            role="City Selection Expert",
            backstory=dedent(f"""Expert at analysing travel data to pick ideal destinations"""),
            goal=dedent(f"""Select the best cities based on weather, season, prices and traveler interests.
                        """),
            tools=[SearchTools.search_internet],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT35,
        )
    
    def local_tour_guide(self):
        return Agent(
            role="Local Tour Guide",
            backstory=dedent(f"""Knowledgeable local guide with extensive information."""),
            goal=dedent(f"""Provide the BEST insights about the selected city"""),
            tools=[SearchTools.search_internet],
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT35,
        )


## Cheatsheet on how to create tasks

In [5]:
'''
Begin with the end in mind. Identify the specific outcome your tasks are aiming to achieve.
- Break down the outcome into actionable tasks, assigning each task to the appropriate agent.
- Ensure that tasks are descriptive, providing clear instructions and expected deliverables.

Goal:
- Develop a detailed itinerary, including city selection, attractions, and practical travel advice

Key steps for task creation:
- Identify the desired outcome, Define what success looks like for the project.
  - A detailed 7 day travel itenerary.

- Task Breakdown: Divide the goal into smaller, manageable tasks that agents can execute
  - Itenerary PLanning: Develop a detailed plan for each day of the trip.
  - City selection: Analyse and pick the best cities to visit.
  - Local Tour Guide: Find a local expert to provide insights and recommendations.

- Assign tasks to agents: Match tasks with agents based on their roles and expertise.

Task description template:
def [Task Name](Self, agent, [parameters]):
    return Task(description=dedent(f
    "
    **Task**: [Provide a concise name or summary of the task]
    **Description**: [Detailed description of what the agent is expected to do, including actionable steps]
    **Parameters**:
    - [Parameter 1]: [Description]
    - [Parameter 2]: [Description]
    "
    , agent=agent))
'''

'\nBegin with the end in mind. Identify the specific outcome your tasks are aiming to achieve.\n- Break down the outcome into actionable tasks, assigning each task to the appropriate agent.\n- Ensure that tasks are descriptive, providing clear instructions and expected deliverables.\n\nGoal:\n- Develop a detailed itinerary, including city selection, attractions, and practical travel advice\n\nKey steps for task creation:\n- Identify the desired outcome, Define what success looks like for the project.\n  - A detailed 7 day travel itenerary.\n\n- Task Breakdown: Divide the goal into smaller, manageable tasks that agents can execute\n  - Itenerary PLanning: Develop a detailed plan for each day of the trip.\n  - City selection: Analyse and pick the best cities to visit.\n  - Local Tour Guide: Find a local expert to provide insights and recommendations.\n\n- Assign tasks to agents: Match tasks with agents based on their roles and expertise.\n\nTask description template:\ndef [Task Name](Sel

In [6]:
from crewai import Task
from textwrap import dedent

class TravelTasks:
    def __tip_section(self):
        return "If you do your BEST WORK, I'll give you a $10,000 commission!"

    def plan_itinerary(self, agent, city, travel_dates, interests):
        return Task(
            description=dedent(
                f"""
            **Task**: Develop a 7 day Travel Itenerary
            **Description**: Expand the city guide into a full 7 day travel itinerary with detailed per day plans,\
            including weather forecasts, places to eat, packing suggestions, and a budget breakdown. 
            You MUST suggest actual places to visit, actual hotels to stay, and actual restaurants to go to.
            This itenerary should cover all aspects of the trip, from arrival to departure,\
            integrating the city guide information with practical travel logistics.
            
            **Parameters**:
            - City: {city}
            - Trip Date: {travel_dates}
            - Traveler interests: {interests}
            
            **Note**: {self.__tip_section()}
        """
            ),
            agent=agent,
        )

    def identify_city(self, agent, origin, cities, interests, travel_dates):
        return Task(
            description=dedent(
                f"""
                **Task**: Identify the Best City for the Trip
                **Description**: Analyze and select the best city for the trip based on specific criteria such as weather patterns,seasonal events, and travel costs.
                This task involves comparing multiple cities, considering factors like current weather conditions, \
                upcoming cultural or seasonal events, and overall travel expenses.
                Your final answer must be a detailed report on the chosen city, including actual flight costs, weather forecast, and attractions.
                
                **Parameters**:
                - Origin: {origin}
                - Cities: {cities}
                - Interests: {interests}
                - Travel Date: {travel_dates}
                
                **Note**: {self.__tip_section()}
        """
            ),
            agent=agent,
        )
        
    def gather_city_info(self, agent, city, interests, travel_dates):
        return Task(
            description=dedent(
                f"""
                **Task**: Gather In-depth City Guide Information
                **Description**: Compile an in-depth guide for the selected city, gathering information about key attractions,\
                local customs, special events, and daily activity recommendations.
                This guide should provide a thorough overview of what the city has to offer, including hidden gems,\
                cultural hotspots, must-visit landmarks, weather forecasts, and high-level cost.
                
                **Parameters**:
                - Cities: {city}
                - Interests: {interests}
                - Travel Date: {travel_dates}
                **Note**: {self.__tip_section()}
        """
            ),
            agent=agent,
        )

## Creating the crew run

In [7]:
from crewai import Crew

class TripCrew:
    def __init__(self, origin, cities, date_range, interests):
        self.origin = origin
        self.cities = cities
        self.date_range = date_range
        self.interests = interests
        
    def run(self):
        # Define the custom agent
        agents = TravelAgents()
        
        # Define custom task
        tasks = TravelTasks()
        
        expert_travel_agent = agents.expert_travel_agent()
        city_selection_expert = agents.city_selection_expert()
        local_tour_guide = agents.local_tour_guide()
        
        plan_itinerary = tasks.plan_itinerary(
            expert_travel_agent,
            self.cities,
            self.date_range,
            self.interests
        )
        
        identify_city = tasks.identify_city(
            city_selection_expert,
            self.origin,
            self.cities,
            self.interests,
            self.date_range
        )

        gather_city_info = tasks.gather_city_info(
            local_tour_guide,
            self.cities,
            self.interests,
            self.date_range
        )
        
        crew = Crew(agents = [expert_travel_agent,
                              city_selection_expert,
                              local_tour_guide
                              ],
                    tasks = [plan_itinerary, identify_city, gather_city_info],
                    verbose=True,
                    )
        result = crew.kickoff()
        return result

In [8]:
print("## Welcome to Trip Planner Crew")
print('-------------------------------')
location = input(
    dedent("""
      From where will you be traveling from?
    """))
cities = input(
    dedent("""
      What are the cities options you are interested in visiting?
    """))
date_range = input(
    dedent("""
      What is the date range you are interested in traveling?
    """))
interests = input(
    dedent("""
      What are some of your high level interests and hobbies?
    """))
  
trip_crew = TripCrew(location, cities, date_range, interests)
result = trip_crew.run()
print("\n\n########################")
print("## Here is you Trip Plan")
print("########################\n")
print(result)

## Welcome to Trip Planner Crew
-------------------------------
[DEBUG]: Working Agent: Expert Travel Agent
[INFO]: Starting Task: 
**Task**: Develop a 7 day Travel Itenerary
**Description**: Expand the city guide into a full 7 day travel itinerary with detailed per day plans,            including weather forecasts, places to eat, packing suggestions, and a budget breakdown. 
You MUST suggest actual places to visit, actual hotels to stay, and actual restaurants to go to.
This itenerary should cover all aspects of the trip, from arrival to departure,            integrating the city guide information with practical travel logistics.

**Parameters**:
- City: Thailand
- Trip Date: August
- Traveler interests: Beaches, Food, sight seeing

**Note**: If you do your BEST WORK, I'll give you a $10,000 commission!



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: I will start by creating a 7 day travel itinerary for a trip to Thailand in August, focu